Here I describe how the dataset was split into training and testing and standardized

In [1]:
import pickle
import warnings

warnings.filterwarnings('ignore', category=RuntimeWarning)

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Split data to training and testing.

In [4]:
df = pd.read_pickle('../DataJar/PickleJar/Interim/df_5_non_standardized.pkl')

In [5]:
df.head()

,log10_etopo2,oisst,solz,PC1,PC2,PC3,PC4,PC5,PC6,log10_aphy411,log10_aphy443,log10_aphy489,log10_aphy510,log10_aphy555,log10_aphy670
2,2.518514,0.78,52.7,0.064018,0.021666,0.005765,0.007146,0.001577,5.577568e-17,-1.617965,-1.511717,-1.686745,-1.899250,-2.414426,-1.982925
3,3.341039,1.79,45.9,0.070175,0.013182,0.000706,0.003175,-0.000535,3.729566e-17,-1.724435,-1.641475,-1.834429,-2.028214,-2.508498,-2.029142
26,3.643749,27.99,13.3,0.184243,-0.011560,0.033405,-0.003724,-0.005967,7.887427e-17,-2.700929,-2.591590,-2.742082,-2.986741,-3.507240,-3.251037
31,1.146128,6.68,27.4,-0.081843,-0.041959,0.000651,-0.005441,-0.004270,1.587916e-17,-0.321900,-0.267589,-0.544316,-0.683378,-1.201066,-0.450383
32,0.954243,10.66,22.2,-0.075830,-0.024480,0.018084,-0.004644,-0.000833,-2.150764e-17,-1.134416,-1.107956,-1.342935,-1.461791,-2.047644,-1.385409


First,  separate predictor (X) from predicted (y)

In [13]:
X = df.dropna().loc[:, :'PC6']

In [14]:
y = df.dropna().filter(regex='aphy')

Split 90%/10% with fixed random state for reproducibility

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.10,
                                                   random_state=42)

Standardization. Note only predictors are standardized here.

In [20]:
scaler = StandardScaler(with_mean=True, with_std=True)

In [28]:
X_train_s = scaler.fit_transform(X_train)

In [29]:
X_train_s

array([[ 1.5922115 ,  0.31637091, -1.18204215, ..., -0.53562835,
         0.27754129,  0.05459972],
       [ 1.69621622, -0.45335793, -0.44955115, ..., -0.05241437,
        -1.14659476,  0.68993956],
       [-0.7879927 , -0.89844923,  0.55423281, ...,  0.83231117,
        -0.23837011, -0.40142711],
       ...,
       [-0.7879927 , -0.95426926,  0.83909042, ...,  0.85142473,
        -0.4727367 , -0.35348952],
       [-0.0373393 ,  0.08280815,  0.25581055, ..., -0.04648142,
        -0.93770504, -0.62854087],
       [ 1.71894864,  0.81434435, -0.55128601, ...,  0.27774257,
        -1.07267356,  0.15492883]])

In [31]:
X_train_s.shape, X_train.shape

((145, 9), (145, 9))

In [33]:
X_train_s = pd.DataFrame(X_train_s, 
                         columns=[f'{col}_s' for col in X_train.columns], 
                         index=X_train.index)

In [35]:
X_test_s = scaler.transform(X_test)
X_test_s = pd.DataFrame(X_test_s, columns=[f'{col}_s' for col in X_test.columns],
                        index=X_test.index)

In [ ]:
datadict = dict(x_train=X_train, x_train_s=X_train_s,
                x_test=X_test, x_test_s=X_test,
                y_train=y_train, y_test=y_test, scaler=scaler)
    
with open('../DataJar/PickleJar/DataSets/AphiTrainTestSplitDataSets_original.pkl',
         'wb') as fb:
    pickle.dump(datadict, fb, protocol=pickle.HIGHEST_PROTOCOL)